In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
# Import necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
from tensorflow.keras.applications import ResNet50
import tensorflow as tf

# Load and Preprocess Data
# Initialize ImageDataGenerator for data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to the range [0, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Set up the training data generator
train_generator = train_datagen.flow_from_directory(
    'train',  # Path to the training data folder (since it's at the same level as README)
    target_size=(150, 150),  # Resize images
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)

# Set up the validation data generator
validation_generator = test_datagen.flow_from_directory(
    'test',  # Path to the test data folder (same as train)
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)




Found 717 images belonging to 6 classes.
Found 0 images belonging to 0 classes.


In [5]:
# Build Transfer Learning Model (Using ResNet50)
# Load the ResNet50 model without the top layers (using pre-trained weights)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional base (ResNet50)
base_model.trainable = False

# Build the model on top of the ResNet50 base
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(5, activation='softmax')  # Output layer for 5 classes (flowers)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model architecture
model.summary()
#GlobalAveragePooling2D: Reduces the output dimensions and captures important features after the ResNet50 convolution layers.
#Dense(5, activation='softmax'): The output layer with 5 nodes corresponding to 5 flower classes.

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 65s 1us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 5, 5, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │           5,125 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,691,013 (98.00 MB)

 Trainable params: 2,103,301 (8.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [6]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Number of steps per epoch
    epochs=10,            # Number of epochs
    validation_data=validation_generator,
    validation_steps=50   # Number of validation steps per epoch
)


C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 6), output.shape=(None, 5)

In [7]:
#Fine-tune the Model
#Now, we unfreeze some of the layers in the ResNet50 base and retrain the model with a smaller learning rate.
# Unfreeze the last few layers of the ResNet50 model for fine-tuning
base_model.trainable = True
fine_tune_at = 100  # Unfreeze layers starting from this index
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile the model with a smaller learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=50
)


Epoch 1/5


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 6), output.shape=(None, 5)

In [8]:
# Save the trained model
model.save('resnet50_finetuned_flower_model.h5')


In [15]:
#Test the Model with New Images
from tensorflow.keras.preprocessing import image
import numpy as np

# Load an image for testing
img_path = 'test/Image_1.jpg'  # Replace with your test image path
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)  # Convert image to array
img_array = img_array / 255.0  # Normalize the image
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make a prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)  # Get predicted class index
print(f'Predicted Class: {train_generator.class_indices.keys()[predicted_class]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


TypeError: 'dict_keys' object is not subscriptable

In [16]:
#Experiment with VGG16 and InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

def build_vgg16_model():
    # Load VGG16 model pre-trained on ImageNet, excluding top layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    
    # Freeze the base model layers
    base_model.trainable = False
    
    # Add custom classification layers
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(5, activation='softmax')  # For 5 flower classes
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [17]:
#Code to Experiment with InceptionV3:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models

def build_inceptionv3_model():
    # Load InceptionV3 model pre-trained on ImageNet, excluding top layers
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    
    # Freeze the base model layers
    base_model.trainable = False
    
    # Add custom classification layers
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(5, activation='softmax')  # For 5 flower classes
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [18]:
# Step 8: Train the Model and Compare Performance

def train_and_evaluate(model, train_generator, validation_generator):
    history = train_model(model, train_generator, validation_generator, epochs=10)
    evaluate_model(model, validation_generator)
    return history

# Create and train models for each architecture
resnet_model = build_model()  # ResNet50
vgg16_model = build_vgg16_model()  # VGG16
inceptionv3_model = build_inceptionv3_model()  # InceptionV3

# Train and evaluate each model
print("Training ResNet50 Model")
history_resnet = train_and_evaluate(resnet_model, train_generator, validation_generator)

print("\nTraining VGG16 Model")
history_vgg16 = train_and_evaluate(vgg16_model, train_generator, validation_generator)

print("\nTraining InceptionV3 Model")
history_inceptionv3 = train_and_evaluate(inceptionv3_model, train_generator, validation_generator)


NameError: name 'build_model' is not defined

In [19]:
# Step 9: Fine-Tune More Layers for Improved Performance
def fine_tune_model_more_layers(model):
    # Unfreeze the base model completely for more fine-tuning
    base_model.trainable = True
    
    # Optionally, unfreeze layers after a certain layer (e.g., layer 100) to reduce computational cost
    # Uncomment the next line if you want to fine-tune from a specific layer
    # for layer in base_model.layers[:100]:
    #     layer.trainable = False
    
    # Re-compile the model with a lower learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    # Continue training the model
    history_fine_tuned = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=5,  # Additional epochs after fine-tuning
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size
    )
    
    return history_fine_tuned


In [20]:
#Code for Aggressive Data Augmentation:
# Step 10: Apply Aggressive Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'flowers/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    'flowers/val',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'flowers/train'

In [21]:
# Step 11: Visualize Training History and Compare Performance

# Function to plot training and validation accuracy
def plot_comparison(history_resnet, history_vgg16, history_inceptionv3):
    plt.figure(figsize=(12, 6))

    # ResNet50
    plt.plot(history_resnet.history['accuracy'], label='ResNet50 Training Accuracy')
    plt.plot(history_resnet.history['val_accuracy'], label='ResNet50 Validation Accuracy')

    # VGG16
    plt.plot(history_vgg16.history['accuracy'], label='VGG16 Training Accuracy')
    plt.plot(history_vgg16.history['val_accuracy'], label='VGG16 Validation Accuracy')

    # InceptionV3
    plt.plot(history_inceptionv3.history['accuracy'], label='InceptionV3 Training Accuracy')
    plt.plot(history_inceptionv3.history['val_accuracy'], label='InceptionV3 Validation Accuracy')

    plt.title('Model Comparison: Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Plot the comparison
plot_comparison(history_resnet, history_vgg16, history_inceptionv3)


NameError: name 'history_resnet' is not defined

In [22]:
#Code to Evaluate Models on Test Data
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Function to evaluate and display performance metrics
def evaluate_performance(model, test_generator):
    # Predict the class probabilities
    predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size)
    predicted_class_indices = np.argmax(predictions, axis=1)
    
    # Get true labels from the generator
    true_class_indices = test_generator.classes
    
    # Generate classification report
    print("Classification Report:")
    print(classification_report(true_class_indices, predicted_class_indices, target_names=test_generator.class_indices.keys()))
    
    # Compute the confusion matrix
    cm = confusion_matrix(true_class_indices, predicted_class_indices)
    
    # Visualize confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
    plt.title('Confusion Matrix')
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    plt.show()

# Evaluate the performance of ResNet50
print("Evaluating ResNet50 Model")
evaluate_performance(resnet_model, test_generator)

# Evaluate the performance of VGG16
print("Evaluating VGG16 Model")
evaluate_performance(vgg16_model, test_generator)

# Evaluate the performance of InceptionV3
print("Evaluating InceptionV3 Model")
evaluate_performance(inceptionv3_model, test_generator)


Evaluating ResNet50 Model


NameError: name 'resnet_model' is not defined

In [23]:
#Code to Fine-Tune the Entire Model:
# Fine-tune the entire model (this might take longer depending on the dataset size)
def fine_tune_entire_model(model, train_generator, validation_generator):
    # Unfreeze all layers in the base model
    for layer in model.layers:
        layer.trainable = True
    
    # Recompile the model with a low learning rate for fine-tuning
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    # Train the model with fine-tuned layers
    history_fine_tuned = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=5,  # Additional fine-tuning epochs
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size
    )
    
    return history_fine_tuned

# Fine-tune ResNet50 (if desired)
history_resnet_finetuned = fine_tune_entire_model(resnet_model, train_generator, validation_generator)

# Fine-tune VGG16 (if desired)
history_vgg16_finetuned = fine_tune_entire_model(vgg16_model, train_generator, validation_generator)

# Fine-tune InceptionV3 (if desired)
history_inceptionv3_finetuned = fine_tune_entire_model(inceptionv3_model, train_generator, validation_generator)


NameError: name 'resnet_model' is not defined

In [24]:
# Save the model
resnet_model.save('resnet50_model.h5')
vgg16_model.save('vgg16_model.h5')
inceptionv3_model.save('inceptionv3_model.h5')

# Load the model back later if needed
# resnet_model = tf.keras.models.load_model('resnet50_model.h5')
# vgg16_model = tf.keras.models.load_model('vgg16_model.h5')
# inceptionv3_model = tf.keras.models.load_model('inceptionv3_model.h5')


NameError: name 'resnet_model' is not defined